[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Zen-Rofiqy/BDC-2024/blob/main/Notebook/BDC.ipynb)

## Imports

In [ ]:
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import re
from sklearn.model_selection import train_test_split
import openpyxl
import matplotlib.pyplot as plt

!pip install swifter
import swifter
from collections import Counter

!pip install translate
from translate import Translator

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import torch
from tqdm.notebook import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=0c128a4b4bf7e158e8f7cc3cbd427683a3a3f7dd6b0d8fad52acf940b7dc52c0
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00


## Prepare Dataset

In [ ]:
# G Drive & Auth
from google.colab import drive, auth

# G Sheet & Auth
from google.auth import default

# Spreadsheet
import gspread as gd

# Connect G Drvie & G Sheet
drive.mount('/content/drive')
auth.authenticate_user()

# Authenticate and initialize gspread
creds, _ = default()
gc = gd.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Open the spreadsheet by URL
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1kYdk_7HDWqYk6X2Gzix9CrU9mynqiZ5KBsWd_EsomSM/edit#gid=0'
workbook = gc.open_by_url(spreadsheet_url)

# Get worksheet titles
sheet_titles = [sheet.title for sheet in workbook.worksheets()]
print(sheet_titles)

['Count', 'Translated', 'dataset_penyisihan_bdc_2024', 'dataset_unlabeled_penyisihan_bdc_2024']


In [ ]:
# Select a specific worksheet
worksheet = workbook.worksheet('Translated')

# Get all data from the worksheet
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])  # First row is header
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.width', None)  # Adjust display width to avoid wrapping

# Display the DataFrame
display(df)

,text,label,TL
0,kunjungan prabowo ini untuk meresmikan dan menyerahkan proyek bantuan air bersih di lima titik indonesiasentris indonesiahijau 02melanjutkan anakmudaindonesiaemas prabowo subianto,Sumber Daya Alam,Prabowo's visit is to inaugurate and hand over the clean water assistance project at five points in Indonesiacentric Indonesiagreen 02continuing young peopleindonesiagold Prabowo Subianto
1,anies dapat tepuk tangan meriah saat jadi rektor mewajibkan mata kuliah anti korupsi untuk memutus mata rantai korupsi aminmiskinkankoruptor,Politik,"Anies received a standing ovation when he became chancellor, requiring anti-corruption courses to break the chain of corruption, amen, poverty, corruption, corruption"
2,memang benar sih pendukung 01 ada yang goblok begitu juga dengan pendukung 02 hnya sj menurut pak ridwan kamil skemanya terbalik kalo 01 mayoritas pendidikan menengah atas (artinya ada juga pendidikan rendah yang milih,Demografi,"It's true that there are supporters of 01 who are stupid, as well as supporters of 02 who only have a bachelor's degree, according to Pak Ridwan Kamil, the scheme is reversed if the majority of 01 have upper secondary education (meaning there are also those with low education who choose"
3,sewaktu anies bersikap kritis ke kinerja pak prabowo dianggap engga sopan karena dianggap kurang menghormati orang tua giliran sekarang gibran yang tengil dan sok kritis malah dianggap kritis dan keras apakah ini tidak standar ganda?,Politik,"When Anies was critical of Pak Prabowo's performance, he was considered impolite because he was deemed to lack respect for parents. Now it's Gibran's turn to be arrogant and critical, but instead he is considered critical and harsh. Is this not a double standard?"
4,anies baswedan harap aparatur sipil negara termasuk tni dan polisi ri pegang sumpahnya dalam pemilu,Politik,"Anies Baswedan hopes that state civil servants, including the Indonesian military and police, will uphold their oath in the election"
...,...,...,...
4995,"melihat debat kemarin tepat prabowo kalah, cepat kekira keluarga yang biasa lakukan aksi kamisan di depan istana : ?",Politik,"Seeing yesterday's debate, Prabowo lost, quickly thought that the family usually held Thursday actions in front of the palace: ?"
4996,masyarakat yakin bahwa prabowo gibran memiliki visi yang sejalan dengan aspirasi rakyat kosongduasatuputaran rakyatsenang 02menang,Politik,people believe that prabowo gibran has a vision that is in line with the aspirations of the people empty two one round people happy 02 win
4997,in my opinion both are irrational but yang satu jauh lebih irrational to be honest but nevertheless dua duanya mesti diluruskan dan baik anies dan ahok harus terlibat langsung untuk validasi trauma mereka dengan empati loh ya gitu ae sih,Ekonomi,"in my opinion both are irrational but one is far more irrational to be honest but nevertheless both must be straightened out and both Anies and Ahok must be directly involved to validate their trauma with empathy, you know, that's how it is"
4998,look at that pak ganjar anda sudah berkecimpung di legislatif eksekutif bahkan calon presiden saja mamat gerindra subianto brani mengancam ancaman itu nyata pak ganjar nyalakan tanda bahaya untuk operasi persenjataan mamat (operasi persenjataan mamat),Pertahanan dan Keamanan,"Look at that Mr Gangar, you have been involved in the executive legislative branch, even as a presidential candidate, Mamat Gerindra Subianto Brani, the threat is real, Mr Gangar has raised the alarm for the Mamat Armament Operation (Mamat Armament Operation)"


## Pre-processing

In [ ]:
# Eksplorasi
Data = df
Data.label.value_counts()

label
Politik                    2972
Sosial Budaya               587
Pertahanan dan Keamanan     400
Ideologi                    400
Ekonomi                     367
Sumber Daya Alam            192
Demografi                    62
Geografi                     20
Name: count, dtype: int64

### Regex

In [ ]:
def remove (text):
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'https\S+', '', text)
    text = re.sub(r'Ã\S+', '', text)
    text = re.sub(r'\[[^\]]*]', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[\,]|[\-]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\?+', '?', text)
    text = re.sub(r'\.', '', text)
    text = re.sub(r'(\w)\1{2,}', r'\1', text)
    cleaned_text = re.sub(r'RT\s', '', text)
    return cleaned_text.strip()

In [ ]:
Clean = Data.copy()
for index, row in Data.iterrows():
    Clean.at[index, "text"] = remove(Data.text[index])
Clean["text"] = Clean["text"].str.lower()

### Normalisasi

In [ ]:
def normalisasi (text, Normal) :
    for i in Normal:
        text = text.replace(i,Normal[i])
    return text

#### Dict

In [ ]:
Normal = {
    "kicep ": "kalah, cepat ",
    "imo ": "in my opinion ",
    "tbh ": "to be honest ",
    "kudu ": "harus ",
    "sdh ": "sudah ",
    "lgislatif ": "legislatif ",
    "bhkan ": "bahkan ",
    "mamatgs ": "mamat gerindra subianto ",
    "mngmcam ": "mengancam ",
    "opm ": "operasi persenjataan mamat ",
    "capres ": "calon presiden ",
    "utk ": "untuk ",
    "unt " : "untuk ",
    " yg " : " yang ",
    "rmh ": "rumah ",
    "emang " : "memang ",
    "emng " : "memang ",
    "emg " : "memang ",
    "ngeliat " : "melihat ",
    "kesejahteran " : "kesejahteraan ",
    "pelangaran " : "pelanggaran ",
    "tak " : "tidak ",
    " gak " : " tidak ",
    "bikin " : "membuat ",
    "cuma " : "hanya ",
    "udh " : "sudah ",
    "bakal " : "akan ",
    "bener " : "benar ",
    "imin " : "muhaimin ",
    "pilpres " : "pemilihan presiden ",
    "cawapres " : "calon wakil presiden ",
    " ga " : " tidak ",
    " ri " : " republik indonesia ",
    "paslon " : "pasangan calon ",
    "kalo " : "jika ",
    "prof " : "profesor ",
    "abah " : "anies ",
    "dng " : "dengan ",
    "pemilu " : "pemilihan umum ",
    "dahulu " : "dulu ",
    "bansos " : "bantuan sosial ",
    "angaran " : "anggaran ",
    "tingi " : "tinggi ",
    "menunjukan " : "menunjukkan ",
    "makin " : "semakin ",
    "hinga " : "hingga ",
    " dg " : " dengan ",
    "tau " : "tahu ",
    " gw " : " saya ",
    "diangap " : "dianggap ",
    "kredit " : "credit ",
    "l3bihbaik " : "lebih baik ",
    "pun " : "walaupun ",
    "menhan " : "kementerian pertahanan ",
    " ia " : " dia ",
    "krn " : "karena ",
    " tp " : " tapi ",
    "utang " : "hutang ",
    " dr " : " dari ",
    "polri " : "polisi ri ",
    " tdk " : " tidak ",
    " gue " : " saya ",
    "kini " : "sekarang ",
    " dgn " : " dengan ",
    " sll " : " selalu ",
    "banga " : "bangga ",
    "bgt " : "banget ",
    "biar " : "supaya ",
    "cewek " : "perempuan ",
    "jabar " : "jawa barat ",
    " jd " : " jadi ",
    "dlm " : "dalam ",
    "sampe " : "sampai ",
    "gua " : "saya ",
    "penghasilanya " : "penghasilan ",
    " nggak " : " tidak ",
    "angota " : "anggota ",
    "pendukungnya " : "pendukung ",
    "pas " : "tepat ",
    "sekedar " : "sekadar ",
    "pake " : "pakai ",
    "liat " : "lihat ",
    "sosmed " : "sosial media ",
    "ungul " : "unggul ",
    "sbg " : "sebagai ",
    "debatcalon " : "debat calon ",
    "hasilnya " : "hasil ",
    " jg " : " juga ",
    "jateng " : "jawa tengah ",
    "aniesbaswedan " : "anies baswedan ",
    "gimana " : "bagaimana ",
    " lu " : " anda ",
    "timnas " : "tim nasional ",
    "berbuatbaik " : "berbuat baik ",
    " lo " : " anda ",
    "bawaslu " : "badan pengawas pemilihan umum",
    "rasanya " : "rasa ",
    "buktinyata " : "bukti nyata ",
    "anaknya " : "anak ",
    "usai " : "setelah ",
    "tergangu " : "terganggu ",
    "debatnya " : "debat ",
    "rekan2 " : "rekan ",
    "jatim " : "jawa timur ",
    "gus " : "muhaimin ",
    "klo " : "kalo ",
    "spt " : "seperti ",
    "milih " : "memilih ",
    "rakyatnya " : "rakyat ",
    "mingu " : "minggu ",
    "skrg " : "sekarang ",
    " bowo " : " prabowo ",
    "asn " : "aparatur sipil negara ",
    "konkrit " : "konkret ",
    " sy " : " saya ",
    "nakes " : "tenaga kesehatan ",
    "anis " : "anies ",
    "abis " : "habis ",
    "org " : "orang ",
    "kemhan " : "kementerian pertahanan ",
    "kementrian" : "kementerian ",
    "knp " : "kenapa ",
    "benar2 " : "benar ",
    "bener2 " : "benar ",
    "orang2 " : "orang ",
    " lho " : " loh ",
    "hektare " : "hektar ",
    "hardnews_politik " : "hardnews politik ",
    "jkt " : "jakarta ",
    "bhw " : "bahwa ",
    "mrk " : "mereka ",
    " enggak " : " tidak ",
    "silakan " : "silahkan ",
    "blm " : "belum ",
    "karna " : "karena ",
    "nyerang " : "serang ",
    " ama " : " sama ",
    " gk " : " tidak ",
    " g " : " tidak ",
    "ojol " : "ojek online ",
    "impor " : "import ",
    " ngga " : " tidak ",
    "wowo " : "prabowo ",
    "bpk " : "bapak ",
    "jgn " : "jangan ",
    "trus " : "terus ",
    "trs " : "terus ",
    "rame " : "ramai ",
    " lg " : " lagi ",
    " bs " : " bisa ",
    " tsb " : " tersebut ",
    "kapasitasnya " : "kapasitas ",
    " usah " : " perlu ",
    "anak2 " : "anak ",
    " ap " : " apa ",
    "capresku " : "calon presiden saya",
    " aku " : " saya ",
    "satgas " : "satuan petugas ",
    "enga " : "tidak ",
    "nkri " : "negara kesatuan republik indonesia ",
    "pdhl " : "padahal ",
    " tu " : " itu ",
    "ekonom " : "ekonomi ",
    " hrs " : " harus ",
    "kemaren " : "kemarin ",
    " tg " : " tentang  ",
    "thn " : "tahun ",
    "mikir " : "pikir ",
    "pilgub " : "pemilihan gubernur ",
    "sm " : "sama ",
    "gampang ": "mudah ",
    "hgu ": "hak guna usaha ",
    "gub ": "gubernur ",
    "menangapi ": "menanggapi ",
    "dengerin ": "dengarkan ",
    "nyoblos ": "memilih ",
    "medsos ": "media sosial ",
    "pinter ": "pintar ",
    "tiba2 ": "tiba ",
    "bkn ": "bukan ",
    "spechless ": "speechless ",
    "pendukung2 ": "pendukung ",
    "kayaknya ": "sepertinya ",
    "unhan ": "universitas pertahanan ",
    "itupun ": "itu pun ",
    "salut ": "hormat ",
    "pengen ": "ingin ",
    "penguasan ": "penguasaan ",
    "ngeri ": "seram ",
    "maksa ": "memaksa ",
    "mending ": "sebaiknya ",
    "cawapresnya ": "calon wakil presidennya ",
    "kempat ": "keempat ",
    "seringkali ": "sering kali ",
    "pangilan ": "panggilan ",
    "fpi ": "front pembela islam ",
    "byk ": "banyak ",
    "pemeratan ": "pemerataan ",
    "negri ": "negeri ",
    "kelakuanya ": "kelakuannya ",
    "dikasih ": "diberi ",
    "bertangung ": "bertanggung ",
    "udh ": "sudah ",
    "disitu ": "disana ",
    "adlh ": "adalah ",
    " rb ": " ribu ",
    "keliatan ": "terlihat ",
    "meningalkan ": "meninggalkan ",
    "pupr ": "pekerjaan umum dan perumahan rakyat ",
    "memanfatkan ": "memanfaatkan ",
    "membludak ": "membeludak ",
    "org2 ": "orang ",
    "terimakasih ": "terima kasih ",
    " kn ": " kan ",
    "ugm ": "universitas gadjah mada ",
    "suficient ": "sufficient ",
    "loe ": "anda ",
    "keknya ": "sepertinya ",
    " kh ": " kah ",
    "abk": "analisis beban kerja ",
    "pengunan ": "penggunaan ",
    "cilik ": "kecil ",
    "naikan ": "naikkan ",
    "sgt ": "sangat ",
    "capresnya ": "calon presidennya ",
    "nyapres ": "mencalonkan diri sebagai presiden ",
    "berasa ": "terasa ",
    "kerjasama ": "kerja sama ",
    "sprti ": "seperti ",
    " mimpin ": " memimpin ",
    "tersingung ": "tersinggung ",
    "kpd ": "kepada ",
    "jawabanya ": "jawabannya ",
    "bukanya ": "bukannya ",
    "denger ": "dengar ",
    "hapy ": "happy ",
    "diangarkan ": "dianggarkan ",
    "sda ": "sumber daya alam ",
    "tanguh ": "tangguh ",
    "uda ": "sudah ",
    "twet ": "tweet ",
    "nendang ": "kuat ",
    "dikatrol ": "diangkat ",
    "bm ": "banyak mau ",
    "mentri ": "menteri ",
    "tunjukan ": "tunjukkan ",
    "hektareatau ": "hektare atau ",
    "mbak ": "kak ",
    "wal ": "walikota ",
    "kayu2an ": "kayu ",
    "prilaku ": "perilaku ",
    "bawa2 ": "bawa ",
    " ad ": " ada ",
    "panjenengan ": "anda ",
    "sdm ": "sumber daya manusia ",
    "aproach ": "approach ",
    "nangis ": "menangis ",
    "thd ": "terhadap ",
    "bohir ": "investor ",
    "vidio ": "video ",
    "cewek ": "perempuan ",
    "sekalian ": "sekaligus ",
    "dng ": "dengan ",
    " kl ": "jika ",
    "dapet ": "dapat ",
    "komen ": "komentar ",
    "kata2 ": "kata ",
    "istora ": "istana olahraga ",
    "sumut ": "sumatra utara ",
    "tengara ": "tenggara ",
    "jaman ": "zaman ",
    "ngaku ": "mengaku ",
    "jabatanya ": "jabatannya ",
    "serem ": "seram ",
    "krna ": "karena ",
    "tetanga ": "tetangga ",
    "kri ": "kapal perang republik indonesia ",
    "drpd ": "daripada ",
    "omonomono ": "omong kosong ",
    "menungu ": "menunggu ",
    "cara2 ": "cara ",
    "ketemu ": "bertemu ",
    "gagasanya ": "gagasannya ",
    "twiter ": "twitter ",
    "teriakan ": "teriakkan ",
    "blg ": "bilang ",
    "perayan ": "perayaan ",
    "alm ": "almarhum ",
    "gimick ": "gimmick ",
    "tinginya ": "tingginya ",
    "tertingal ": "tertinggal ",
    "opresed ": "oppressed ",
    "kelar ": "selesai ",
    "kek ": "seperti ",
    " elu ": " anda ",
    " kamu " : " anda ",
    "disat ": "disaat ",
    "sblm ": "sebelum ",
    " pol ": " polisi ",
    "feb ": "februari ",
    "relawanya ": "relawannya ",
    "muhamadiyah ": "muhammadiyah ",
    "ordal ": "orang dalam ",
    "dishub ": "dinas perhubungan ",
    "kaltim ": "kalimantan timur ",
    "pertanyanya ": "pertanyaannya ",
    "jadinya ": "akibatnya ",
    "jalanya ": "jalannya ",
    "stlh ": "setelah ",
    "maling ": "pencuri ",
    "caleg ": "calon legislatif ",
    "kejayan ": "kejayaan ",
    "pwnu ": "pengurus wilayah nahdlatul ulama ",
    "melek ": "mengerti ",
    "bagi2 ": "bagi ",
    "terang2an ": "secara terbuka ",
    "supya ": "agar ",
    "plusnya ": "kelebihannya ",
    "wanita ": "perempuan ",
    "hajatan ": "perayaan",
    "ketidakpercayan ": "ketidakpercayaan ",
    "curhat ": "cerita ",
    "smkn ": "semakin ",
    "cowok ": "pria ",
    " jt ": " juta ",
    " klu ": " klasifikasi lapangan usaha ",
    "takan ": "tidak akan ",
    "pantura ": "pantai utara ",
    "gabung " : "bergabung ",
    " jkw ": " jokowi ",
    "kemampuanya " : "kemampuannya ",
    "gama ": "gamma ",
    "lansia ": "lanjut usia ",
    " msh ": " masih ",
    "modelan ": "tiruan ",
    " lbh ": " lebih ",
    " nt ": " nice try ",
    "kunjunganya ": "kunjungannya ",
    "yth ": "yang terhormat ",
    "terangan ": "terbuka ",
    "sangup ": "sanggup ",
    "faham ": "paham ",
    "bapa ": "bapak ",
    "pingir ": "pinggir ",
    "pkl ": "praktik kerja lapangan ",
    "wil ": "wilayah ",
    "skrng ": "sekarang ",
    "simpel ": "sederhana ",
    "keungulan ": "keunggulan ",
    " cn ": " cina ",
    "ngurus ": "urus ",
    "tangapan ": "tanggapan ",
    "lagian ": "lagi pula ",
    "akn ": "akademi komunitas negeri ",
    "tantrum ": "mengamuk ",
}





Clean["text"]=Clean["text"].apply(lambda x:normalisasi(x,Normal))

### Splitter

In [ ]:
#Tokenisasi
Tokenized = Clean["text"].apply(lambda x:x.split())
Tokenized

0                                                                                              [kunjungan, prabowo, ini, untuk, meresmikan, dan, menyerahkan, proyek, bantuan, air, bersih, di, lima, titik, indonesiasentris, indonesiahijau, 02melanjutkan, anakmudaindonesiaemas, prabowo, subianto]
1                                                                                                                                      [anies, dapat, tepuk, tangan, meriah, saat, jadi, rektor, mewajibkan, mata, kuliah, anti, korupsi, untuk, memutus, mata, rantai, korupsi, aminmiskinkankoruptor]
2                                        [mmemang, benar, sih, pendukung, 01, ada, yang, goblok, begitu, juga, dengan, pendukung, 02, hnya, sj, menurut, pak, ridwan, kamil, skemanya, terbalik, jika, 01, mayoritas, pendidikan, menengah, atas, (artinya, ada, juga, pendidikan, rendah, yang, milih]
3                         [sewaktu, anies, bersikap, kritis, ke, kinerja, pak, prabowo, dianggap, engga, sopan, 

In [ ]:
# Create a new worksheet
workbook.add_worksheet(title='Clean', rows=10000, cols=20)
worksheet = workbook.worksheet('Clean')

# Convert DataFrame to list of lists
data_to_write = [Clean.columns.values.tolist()] + Clean.values.tolist()

# Update the new worksheet with data
worksheet.update('A1', data_to_write)

<ipython-input-40-70db6f4bcafb>:9: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('A1', data_to_write)


{'spreadsheetId': '1kYdk_7HDWqYk6X2Gzix9CrU9mynqiZ5KBsWd_EsomSM',
 'updatedRange': 'Clean!A1:C5001',
 'updatedRows': 5001,
 'updatedColumns': 3,
 'updatedCells': 15003}

### Penerjemahan

In [ ]:
Inggris = Clean.copy()
def translate(text):
    translator = Translator(to_lang="en", from_lang="id")
    translation = translator.translate(text)
    print("done")
    return translation

Inggris["text"] = Clean["text"].apply(translate)

done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


KeyboardInterrupt: 

In [ ]:
# Create a new worksheet
workbook.add_worksheet(title='Inggris', rows=10000, cols=20)
worksheet = workbook.worksheet('Inggris')

# Convert DataFrame to list of lists
data_to_write = [Inggris.columns.values.tolist()] + Inggris.values.tolist()

# Update the new worksheet with data
worksheet.update('A1', data_to_write)

### Steming

In [ ]:
 #Stemming
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean

stem = Tokenized.apply(stemming)
stem.to_excel("C:/Users/ASUS/OneDrive/Kampus IPB/Lomba/BDC/BDC 2024/Data/stem.xlsx", index = False)

## Pemodelan

### Bert

#### Pre-process

In [ ]:
Translated = df
Translated
Translated["text"]=Translated["TL"]
Data_tr = Translated[["text","label"]].copy()
Data_tr["text"] = Data_tr["text"].str.lower()
Data_tr["id"] = range(0,Data_tr.shape[0])

In [ ]:
label_code = {
    "Politik" : 0,
    "Sosial Budaya" : 1,
    "Pertahanan dan Keamanan" : 2,
    "Ideologi" : 3,
    "Ekonomi" : 4,
    "Sumber Daya Alam" : 5,
    "Demografi" : 6,
    "Geografi" : 7

}
Data_tr["Angka"] = Data_tr.label.replace(label_code)
Data_tr['data_type'] = ['not_set'] * Data_tr.shape[0]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(Data_tr.text.values,
                                                Data_tr.Angka.values,
                                                test_size = 0.2,
                                                random_state = 2324,
                                                stratify = Data_tr.Angka.values)
Data_tr['data_type'] = ['not_set'] * Data_tr.shape[0]
Data_tr.head()

,text,label,id,Angka,data_type
0,prabowo's visit is to inaugurate and hand over the clean water assistance project at five points in indonesiacentric indonesiagreen 02continuing young peopleindonesiagold prabowo subianto,Sumber Daya Alam,0,5,not_set
1,"anies received a standing ovation when he became chancellor, requiring anti-corruption courses to break the chain of corruption, amen, poverty, corruption, corruption",Politik,1,0,not_set
2,"it's true that there are supporters of 01 who are stupid, as well as supporters of 02 who only have a bachelor's degree, according to pak ridwan kamil, the scheme is reversed if the majority of 01 have upper secondary education (meaning there are also those with low education who choose",Demografi,2,6,not_set
3,"when anies was critical of pak prabowo's performance, he was considered impolite because he was deemed to lack respect for parents. now it's gibran's turn to be arrogant and critical, but instead he is considered critical and harsh. is this not a double standard?",Politik,3,0,not_set
4,"anies baswedan hopes that state civil servants, including the indonesian military and police, will uphold their oath in the election",Politik,4,0,not_set
